## Mapa

In [12]:
import pandas as pd
import folium

# Ruta al archivo JSON
file_path = "dataset/transacciones_actualizadas.json"

# Cargar el JSON como lista
try:
    df = pd.read_json(file_path)  # No usar `lines=True`
    print("Datos cargados exitosamente.")
except ValueError as e:
    print(f"Error al cargar el archivo JSON: {e}")
    raise

# Verificar si el dataset tiene datos
if df.empty:
    raise ValueError("El archivo JSON no contiene datos o está vacío.")

# Crear un mapa centrado en las coordenadas promedio del dataset
centro_latitud = df["latitud"].mean()
centro_longitud = df["longitud"].mean()
mapa = folium.Map(location=[centro_latitud, centro_longitud], zoom_start=12)

# Añadir marcadores para cada transacción en el dataset
for _, fila in df.iterrows():
    popup_info = f"<b>ID Transacción:</b> {fila['id_transaccion']}<br>" \
                 f"<b>Monto:</b> {fila['monto']}<br>" \
                 f"<b>Ordenante:</b> {fila['ordenante_nombre_completo']}<br>" \
                 f"<b>Beneficiario:</b> {fila['beneficiario_nombre_completo']}"

    folium.Marker(
        location=[fila["latitud"], fila["longitud"]],
        popup=folium.Popup(popup_info, max_width=300),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
output_path = "mapa_transacciones.html"
mapa.save(output_path)
print(f"Mapa interactivo guardado en: {output_path}")


Datos cargados exitosamente.
Mapa interactivo guardado en: mapa_transacciones.html


## Preprocesamiento y normalizacion del dataset

In [13]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Cargar el dataset desde un archivo JSON
file_path = r"dataset/transacciones_actualizadas.json"
df = pd.read_json(file_path)

# Filtrar las columnas relevantes
variables_interes = ["ordenante_cedula", "latitud", "longitud", "fecha_hora", "local_nombre"]
df = df[variables_interes]

# Completar valores faltantes en 'local_nombre'
df["local_nombre"] = df["local_nombre"].fillna("Desconocido")

# Guardar las columnas originales de latitud y longitud
df["latitud_original"] = df["latitud"]
df["longitud_original"] = df["longitud"]

# Extraer hora y día de la semana de 'fecha_hora'
df["hora"] = pd.to_datetime(df["fecha_hora"]).dt.hour
df["dia_semana"] = pd.to_datetime(df["fecha_hora"]).dt.weekday

# Eliminar la columna original 'fecha_hora' ya que ahora está representada
df = df.drop(columns=["fecha_hora"])

# Codificar 'local_nombre' con LabelEncoder para convertirlo en una variable numérica
label_encoder = LabelEncoder()
df["local_nombre_encoded"] = label_encoder.fit_transform(df["local_nombre"])

# Mostrar el dataset transformado
print("Dataset transformado con columnas originales de latitud y longitud:")
print(df.head())

# Guardar el dataset transformado
output_path = "dataset/transacciones_transformadas.json"
df.to_json(output_path, orient="records", lines=True)
print(f"Dataset transformado guardado en: {output_path}")

Dataset transformado con columnas originales de latitud y longitud:
   ordenante_cedula    latitud   longitud      local_nombre  latitud_original  \
0        3597400581 -12.046357 -77.042883       Desconocido        -12.046357   
1        3597400581 -12.046464 -77.042727       Desconocido        -12.046464   
2        3597400581 -12.046440 -77.042898       Desconocido        -12.046440   
3        3597400581  -3.995926 -79.204172  Bogati Heladeria         -3.995926   
4        3597400581  -4.007181 -79.203194       Magic Retro         -4.007181   

   longitud_original  hora  dia_semana  local_nombre_encoded  
0         -77.042883     2           6                     3  
1         -77.042727     0           2                     3  
2         -77.042898    15           3                     3  
3         -79.204172     7           3                     0  
4         -79.203194    18           3                     7  
Dataset transformado guardado en: dataset/transacciones_transformad

## Dividir el dataset para entrenamiento y para testeo

In [14]:
from sklearn.model_selection import train_test_split
import pandas as pd

    # Cargar el dataset transformado
file_path = "dataset/transacciones_transformadas.json"
df = pd.read_json(file_path, lines=True)

# Dividir los datos por usuario (ordenante_cedula)
usuarios = df["ordenante_cedula"].unique()

# Crear conjuntos de usuarios para entrenamiento y testeo
usuarios_train, usuarios_test = train_test_split(
        usuarios, test_size=0.2, random_state=42
    )

    # Separar el dataset en base a los usuarios seleccionados
df_train = df[df["ordenante_cedula"].isin(usuarios_train)]
df_test = df[df["ordenante_cedula"].isin(usuarios_test)]

    # Guardar los datasets en archivos separados
train_path = "dataset/transacciones_train.json"
test_path = "dataset/transacciones_test.json"

df_train.to_json(train_path, orient="records", lines=True)
df_test.to_json(test_path, orient="records", lines=True)

    # Imprimir mensajes de confirmación
print(f"Dataset de entrenamiento guardado en: {train_path}")
print(f"Dataset de testeo guardado en: {test_path}")

    # Verificar las divisiones con ejemplos
print("Ejemplo del dataset de entrenamiento:")
print(df_train.head())
print("\nEjemplo del dataset de testeo:")
print(df_test.head())

Dataset de entrenamiento guardado en: dataset/transacciones_train.json
Dataset de testeo guardado en: dataset/transacciones_test.json
Ejemplo del dataset de entrenamiento:
     ordenante_cedula    latitud   longitud      local_nombre  \
100        4557997274  -3.995926 -79.204172  Bogati Heladeria   
101        4557997274 -12.046304 -77.042721       Desconocido   
102        4557997274 -12.046434 -77.042738       Desconocido   
103        4557997274 -12.046445 -77.042892       Desconocido   
104        4557997274 -12.046400 -77.042766       Desconocido   

     latitud_original  longitud_original  hora  dia_semana  \
100         -3.995926         -79.204172    10           1   
101        -12.046304         -77.042721     1           1   
102        -12.046434         -77.042738     9           0   
103        -12.046445         -77.042892    21           5   
104        -12.046400         -77.042766     4           0   

     local_nombre_encoded  
100                     0  
101     

## Creacion de Modelo KNN

In [15]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import joblib

# Cargar el dataset transformado y normalizado
train_path = "dataset/transacciones_train.json"
df_train = pd.read_json(train_path, lines=True)

# Definir los pesos para las características
pesos = {
    "latitud": 0.3,
    "longitud": 0.3,
    "hora": 0.3,
    "dia_semana": 0.1
}

# Aplicar los pesos a las características normalizadas
for columna, peso in pesos.items():
    df_train[columna] *= peso

# Seleccionar las características para entrenar el modelo
X_train = df_train[["latitud", "longitud", "hora", "dia_semana"]]

# Entrenar el modelo KNN
modelo_knn = NearestNeighbors(n_neighbors=10, metric="euclidean")
modelo_knn.fit(X_train)

NearestNeighbors(metric='euclidean', n_neighbors=10)

In [16]:
# Guardar el modelo entrenado
joblib.dump(modelo_knn, "recomendador_knn.pkl")

print("Modelo KNN entrenado y guardado correctamente.")


Modelo KNN entrenado y guardado correctamente.


## Función para Recomendar

In [20]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic

# Pesos para las características (aplicados antes de usar KNN)
pesos = {
    "latitud": 0.3,
    "longitud": 0.3,
    "hora": 0.3,
    "dia_semana": 0.1
}

# Cargar el modelo KNN previamente entrenado
modelo_knn = joblib.load("recomendador_knn.pkl")

def recomendar_por_cedula(cedula, cliente, radio_haversine=10.0, n_neighbors=5):
    """
    Realiza recomendaciones basadas en el historial de transacciones del usuario.
    :param cedula: Cédula del usuario.
    :param cliente: Lista con las características del cliente (latitud, longitud, hora, día de la semana).
    :param radio_haversine: Radio en kilómetros para filtrar recomendaciones.
    :param n_neighbors: Número de vecinos a considerar con KNN.
    :return: Recomendaciones de comercios dentro del radio.
    """
    try:
        # Cargar el dataset de entrenamiento
        train_path = "dataset/transacciones_train.json"
        df_train = pd.read_json(train_path, lines=True)

        # Filtrar el historial del usuario por cédula
        historial_usuario = df_train[df_train["ordenante_cedula"] == cedula]
        if historial_usuario.empty:
            print(f"No se encontró historial para el usuario con cédula: {cedula}")
            return None

        # Aplicar los pesos a los datos del cliente
        cliente_datos = [
            cliente[0] * pesos["latitud"],
            cliente[1] * pesos["longitud"],
            cliente[2] * pesos["hora"],
            cliente[3] * pesos["dia_semana"]
        ]

        # Obtener los vecinos más cercanos utilizando KNN
        X_train = historial_usuario[["latitud", "longitud", "hora", "dia_semana"]].copy()
        for columna, peso in pesos.items():
            X_train[columna] *= peso
        modelo_knn.fit(X_train)  # Entrenar el modelo con el historial del usuario
        distancias, indices = modelo_knn.kneighbors([cliente_datos], n_neighbors=n_neighbors)

        # Recuperar las recomendaciones iniciales
        recomendaciones = historial_usuario.iloc[indices[0]].copy()

        # Calcular la distancia Haversine utilizando las columnas originales
        cliente_coords = (cliente[0], cliente[1])  # Coordenadas originales del cliente
        recomendaciones["distancia_haversine"] = recomendaciones.apply(
            lambda row: geodesic(
                (row["latitud_original"], row["longitud_original"]),
                cliente_coords
            ).kilometers,
            axis=1
        )

        # Filtrar las recomendaciones dentro del radio Haversine
        recomendaciones_filtradas = recomendaciones[recomendaciones["distancia_haversine"] <= radio_haversine]

        if recomendaciones_filtradas.empty:
            print(f"No se encontraron recomendaciones dentro del radio de {radio_haversine} km.")
            return None

        # Retornar las recomendaciones finales
        return recomendaciones_filtradas[[
            "local_nombre", "latitud_original", "longitud_original", 
            "hora", "dia_semana", "distancia_haversine"
        ]]

    except Exception as e:
        print(f"Error en la función de recomendación: {str(e)}")
        return None

# Datos del cliente: características originales
cedula_cliente = 4557997274
cliente_datos = [-3.995926, -79.204172, 10, 1]  # Coordenadas originales y contexto del cliente

# Radio Haversine en kilómetros
radio_haversine = 10

# Realizar la recomendación
recomendaciones = recomendar_por_cedula(cedula_cliente, cliente_datos, radio_haversine=radio_haversine)

# Imprimir las recomendaciones
if recomendaciones is not None:
    print("Recomendaciones finales:")
    print(recomendaciones)


Recomendaciones finales:
            local_nombre  latitud_original  longitud_original  hora  \
0       Bogati Heladeria         -3.995926         -79.204172    10   
37  Manantial restaurant         -4.000665         -79.201764    10   
48           Magic Retro         -4.007181         -79.203194    10   
41   Las Papas del hueco         -3.994721         -79.202306    11   
31        Chifa Oriental         -4.000614         -79.204941    11   

    dia_semana  distancia_haversine  
0            1             0.000032  
37           3             0.588340  
48           3             1.249289  
41           3             0.246393  
31           3             0.525382  


c:\Practicum2.1\venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
